In [158]:
from array import array
import math
import reprlib
import operator
import functools
import itertools

class Vector:
    typecode = 'd'
    
    __match_args__ = ('x', 'y', 'z', 't')
    
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
    
    # Iterable protocol
    def __iter__(self):
        return iter(self._components)
    
    # Sequence protocol
    def __len__(self):
        return len(self._components)
    
    # Sequence protocol
    def __getitem__(self, key):
        if isinstance(key, slice):
            cls = type(self)
            return cls(self._components[key])
        
        index = operator.index(key)
        return self._components[key]
        
    
    def __getattr__(self, attr):
        cls = type(self)
        try:
            pos = cls.__match_args__.index(attr)
        except ValueError:
            pos = -1
        
        if 0 <= pos <= len(self._components):
            print('are you here??')
            return self._components[pos]
        
        msg = f'{cls.__name__!r} object has no attribute {attr!r}'
        raise AttributeError(msg)

    def __repr__(self):
        class_name = type(self).__name__
        components = reprlib.repr(self._components)
        
        components = components[components.find('['):-1]
        
        return f'{class_name}({components})'

    def __str__(self):
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + (bytes(self._components)))
    
    def __eq__(self, other):
        if len(self) != len(other):
            return False
        return all(a == b for a,b in zip(self, other))

    def __hash__(self):
        hashes = (hash(x) for x in self._components)
        return functools.reduce(operator.xor, hashes, 0)

    def __abs__(self):
        return math.hypot(*self)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('h'):
            fmt_spec = fmt_spec[:-1]
            coords = itertools.chain([abs(self)], self.angles())
            outer_fmt = '<{}>'
        else:
            coords = self
            outer_fmt = '({})'
            
        components = (format(c, fmt_spec) for c in coords)
        
        return outer_fmt.format(', '.join(components))
    
    @classmethod
    def from_bytes(cls, octets):
        cls.typecode = chr(octets[0])
        
        memv = memoryview(octets[1:]).cast(cls.typecode)
        
        return cls(memv)
    
    def angle(self, n):
        r = math.hypot(*self[n:])
        a = math.atan2(r, self[n-1])
        if (n == len(self) - 1) and (self[-1] < 0):
            return math.pi * 2 - a
        else:
            return a
    
    def angles(self):
        return (self.angle(n) for n in range(1, len(self)))

v = Vector([1,2,3,4,5])

v.x = 7
print(v.x)
print(v)

format(v, '0.5fh')

7
(1.0, 2.0, 3.0, 4.0, 5.0)


'<7.41620, 1.43554, 1.29515, 1.13265, 0.89606>'

In [147]:
import operator

print(operator.xor(12345, 5434))

9475


In above example `v.x` is set on `v` as an instance attribute. When we retrieve it through `v.x` it prints the updated value.

However, when printing the whole vector only `self._components` are taken into account, so `x` instance attribute is skipped.